# Comparing Two Climate Models

In this notebook, I will be comparing two climate reanalysis models:

* NCEP-DOE Reanalysis 2: Surface
* ERA5

I will be looking at the following variables:

* Surface Pressure
* Mean Sea Level Pressure
* Total Column Water

The idea is simple: these two models should have very similar properties. I will be trying to user RBIG in order to assess how similar these models are. I'll be looking at the following IT measures

* Entropy
* Total Correlation
* Mutual Information

If these climate models are that similar, then they should exhibit similar IT measures.

## Data - Climate Models

In [190]:
import os, sys
cwd = os.getcwd()
source_path = f"{cwd}/../../../"
sys.path.insert(0, f'{source_path}')

import numpy as np

# Data Loaders
from src.data.climate.amip import DataDownloader
from src.data.climate.amip import DataLoader
from src.data.climate.era5 import get_era5_data
from src.data.climate.ncep import get_ncep_data
from src.features.climate.build_features import (
    get_time_overlap, check_time_coords, regrid_2_lower_res, get_spatial_cubes, normalize_data)


# Stat Tools
from src.models.train_models import run_rbig_models
from scipy import stats

import pandas as pd
import xarray as xr
from tqdm import tqdm
from sklearn import preprocessing

import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
amip_data_path = f"/home/emmanuel/projects/2020_rbig_rs/data/climate/raw/amip/"
era5_path = f"/home/emmanuel/projects/2020_rbig_rs/data/climate/raw/era5/"
ncep_path = f"/home/emmanuel/projects/2020_rbig_rs/data/climate/raw/ncep/"
results_path = f"/home/emmanuel/projects/2020_rbig_rs/data/climate/results/"
fig_path = f"/home/emmanuel/projects/2020_rbig_rs/reports/figures/climate/"

## Demo Experiment

### Experimental Paams

In [208]:
class DataArgs:
    data_path = "/home/emmanuel/projects/2020_rbig_rs/data/climate/raw/amip/"
    results_path = "/home/emmanuel/projects/2020_rbig_rs/data/climate/results/amip"

class CMIPArgs:

        
    # Fixed Params
    spatial_windows = [
        1, 2,                # Spatial Window for Density Cubes
        3,4,5,6,7,8,9,10
    ]

    # Free Params
    variables = [
        'psl'               # Mean Surface Pressure
    ]
    cmip_models = [
        "inmcm4"
    ]
    
    base_models = [
        'ncep',
    ]
        

In [177]:
def run_exp():
    
    for ibase in CMIPArgs.base_models:
        for ivariable in CMIPArgs.variables:
            for icmip in CMIPArgs.cmip_models:
                print(ibase)
                print(ivariable)
                print(icmip)

In [178]:
run_exp()

ncep
psl
inmcm4


### Part I - Grab Data

In [179]:
from typing import Type, Union

xr_types = Union[xr.Dataset, xr.DataArray]

def get_base_model(base_model: str, variable: str)-> xr_types:
    
    if base_model == 'era5':
        ds = get_era5_data()[variable]
    elif base_model == 'ncep':
        ds = get_ncep_data()[variable]
    else:
        raise ValueError("Unrecognized base model:", base_model)
    ds.attrs['model_id'] = base_model
    return ds


def get_cmip5_model(cmip_model: str, variable: str)-> xr_types:
    
    loader = DataLoader()
    
    ds = loader.load_amip_data(cmip_model)[variable]
    ds.attrs['model_id'] = cmip_model
    return ds

In [180]:
base_dat = get_base_model(CMIPArgs.base_models[0], CMIPArgs.variables[0])
base_dat

<xarray.DataArray 'psl' (time: 489, lat: 73, lon: 144)>
dask.array<open_dataset-0ace0936f02ce97f3d79321b6d5f6a55mslp, shape=(489, 73, 144), dtype=float32, chunksize=(489, 73, 144), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2019-09-01
Attributes:
    long_name:             Monthly Mean Sea Level Pressure
    valid_range:           [-32766  15234]
    unpacked_valid_range:  [ 77000. 125000.]
    actual_range:          [ 95644. 105703.]
    units:                 Pascals
    precision:             0
    GRIB_id:               2
    GRIB_name:             PRMSL
    var_desc:              Mean Sea Level Pressure
    dataset:               NCEP/DOE AMIP-II Reanalysis (Reanalysis-2) Monthly...
    level_desc:            Sea Level
    statistic:             Individual Obs
    parent_s

In [181]:
cmip_dat = get_cmip5_model(CMIPArgs.cmip_models[0], CMIPArgs.variables[0])
cmip_dat

<xarray.DataArray 'psl' (time: 360, lat: 120, lon: 180)>
dask.array<open_dataset-0a95b28dbc539d100c816bea9e2db9e7psl, shape=(360, 120, 180), dtype=float32, chunksize=(360, 120, 180), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 1979-01-16 12:00:00 ... 2008-12-16 12:00:00
  * lat      (lat) float64 -89.25 -87.75 -86.25 -84.75 ... 86.25 87.75 89.25
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
Attributes:
    standard_name:     air_pressure_at_sea_level
    long_name:         Sea Level Pressure
    comment:           not, in general, the same as surface pressure
    units:             Pa
    original_name:     psl
    cell_methods:      time: mean (interval: 1 month)
    history:           2011-02-07T11:53:19Z altered by CMOR: Reordered dimens...
    associated_files:  baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation...
    model_id:          inmcm4

In [182]:
def run_exp():
    
    for ibase in CMIPArgs.base_models:
        for ivariable in CMIPArgs.variables:
            for icmip in CMIPArgs.cmip_models:
                print(ibase)
                print(ivariable)
                print(icmip)
                
                

### Part II - Regrid Data

In [183]:
base_dat, cmip_dat = regrid_2_lower_res(base_dat, cmip_dat)

assert(base_dat.shape[1] == cmip_dat.shape[1])
assert(base_dat.shape[2] == cmip_dat.shape[2])
base_dat

Create weight file: nearest_s2d_120x180_73x144.nc
Remove file nearest_s2d_120x180_73x144.nc


<xarray.DataArray 'psl' (time: 489, lat: 73, lon: 144)>
dask.array<open_dataset-0ace0936f02ce97f3d79321b6d5f6a55mslp, shape=(489, 73, 144), dtype=float32, chunksize=(489, 73, 144), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2019-09-01
Attributes:
    long_name:             Monthly Mean Sea Level Pressure
    valid_range:           [-32766  15234]
    unpacked_valid_range:  [ 77000. 125000.]
    actual_range:          [ 95644. 105703.]
    units:                 Pascals
    precision:             0
    GRIB_id:               2
    GRIB_name:             PRMSL
    var_desc:              Mean Sea Level Pressure
    dataset:               NCEP/DOE AMIP-II Reanalysis (Reanalysis-2) Monthly...
    level_desc:            Sea Level
    statistic:             Individual Obs
    parent_s

### Part III - Find Overlapping Times

In [184]:
base_dat.shape, cmip_dat.shape

((489, 73, 144), (360, 73, 144))

In [185]:
base_dat, cmip_dat = get_time_overlap(base_dat, cmip_dat)

### Part IV - Groupby time stamp

In [186]:
time_stamps = min(len(base_dat.time), len(cmip_dat.time))

with tqdm(range(time_stamps)) as progress_bar:
    for itime in progress_bar:
        ibase_dat = base_dat.isel(time=itime, drop=False)
        icmip_dat = cmip_dat.isel(time=itime)
        print(ibase_dat)
        print(icmip_dat)
        break


  0%|          | 0/359 [00:00<?, ?it/s]

<xarray.DataArray 'psl' (lat: 73, lon: 144)>
dask.array<getitem, shape=(73, 144), dtype=float32, chunksize=(73, 144), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
    time     datetime64[ns] 1979-02-01
Attributes:
    long_name:             Monthly Mean Sea Level Pressure
    valid_range:           [-32766  15234]
    unpacked_valid_range:  [ 77000. 125000.]
    actual_range:          [ 95644. 105703.]
    units:                 Pascals
    precision:             0
    GRIB_id:               2
    GRIB_name:             PRMSL
    var_desc:              Mean Sea Level Pressure
    dataset:               NCEP/DOE AMIP-II Reanalysis (Reanalysis-2) Monthly...
    level_desc:            Sea Level
    statistic:             Individual Obs
    parent_stat:           Other
    standard_name:         pressure
    cell_methods:          time: mean (

### Part IV - Get Density Cubes

In [209]:
base_df = get_spatial_cubes(ibase_dat.expand_dims({"time": 1}), CMIPArgs.spatial_windows[3])
cmip_df = get_spatial_cubes(icmip_dat.expand_dims({"time": 1}), CMIPArgs.spatial_windows[3])

### Part V - Normalize

In [214]:
base_norm = normalize_data(base_df)
cmip_norm = normalize_data(cmip_df)

(9870, 16)

### Part VI - RBIG Algorithm

#### Entropy, Total Correlation

In [211]:
base_tc, base_h, _ = run_rbig_models(base_norm, measure='t', verbose=None)
cmip_tc, cmip_h, _ = run_rbig_models(cmip_norm, measure='t', verbose=None)

In [215]:
base_tc/16, cmip_tc/16

(3.15210921530442, 2.898822929574796)

In [216]:
base_h/16, cmip_h/16

(-1.9835297431860046, -1.6207488092653572)

#### Mutual Information

In [217]:
mi, t_ = run_rbig_models(base_norm, cmip_norm, measure='mi', verbose=None)

In [219]:
mi

1.7665437827063366

## Experiment I - Comparing Climate Models

### Mean Sea Level Pressure

#### ERA5 vs NCEP

In [12]:
# Experiment class
class ClimateEntropy:
    def __init__(self, save_path: None, variable: str='mslp', save_name=None, mi: bool=True):
        
        self.variable = variable
        
        self.results_path = save_path
        self.results_df = pd.DataFrame()
        self.save_name = save_name
        self.mi = mi
    
    def run_experiment(self, climate_model1: pd.DataFrame, climate_model2: pd.DataFrame) -> None:
        """Training loop that goes through each year and calculates the entropy,
        total correlation and mutual information between the two models."""
        time_length = len(climate_model1.groupby('time.year'))
        
        # Normalize BEFORE the individual calculations
        climate_model1[self.variable] = normalize_temporal(climate_model1[self.variable])
        
        model1_id = climate_model1.attrs['model_id']
        model2_id = climate_model2.attrs['model_id']
        climate_model2[self.variable] = normalize_temporal(climate_model2[self.variable])
        
        with tqdm(zip(
            climate_model1.groupby('time.year'), 
            climate_model2.groupby('time.year')
        ), total=time_length) as progress_bar:
            
            for imodel1, imodel2 in progress_bar:
                
                # Update params in progress bar
                
                # Transform to dataframe, remove spatial dimensions
                X1 = self._get_time_features(imodel1[1][self.variable])
                X2 = self._get_time_features(imodel2[1][self.variable])
                
                # Normalize inputs
                min_max_scaler = preprocessing.StandardScaler()
                X1 = min_max_scaler.fit_transform(X1.values)
                X2 = min_max_scaler.fit_transform(X2.values)
                
                dims = X1.shape[1]
                
                # =============================
                # Calculate Mutual Information
                # =============================
                if self.mi == False:
                    mi_ = None
                    mi_t_ = None
                else:
                    mi_, mi_t_ = run_rbig_models(X1, X2, measure='mi', verbose=None)
                
                
                # Update params in progress bar
                postfix = dict(
                    
                )

                # ========================================
                # Calculate Entropy and Total Correlation
                # ========================================
                
                # Model I
                tc1_, h1_, h_t1_ = run_rbig_models(X1, measure='t', verbose=None)
                
                self._update_results(
                    model=model1_id,
                    year=imodel1[0],
                    h_time=h_t1_,
                    tc=tc1_,
                    h=h1_,
                    mi=mi_,
                    mi_time=mi_t_,
                    dims=dims,
                    
                )
                
                
                # Model II
                tc2_, h2_, h_t2_ = run_rbig_models(X2, measure='t', verbose=None)
                self._update_results(
                    model=model2_id,
                    year=imodel2[0],
                    h_time=h_t2_,
                    tc=tc2_,
                    h=h2_,
                    mi=mi_,
                    mi_time=mi_t_,
                    dims=dims,
                    
                )
                
                # Update params in progress bar
                postfix = dict(
                    year=imodel1[0],
                    mi=f"{mi_:.3f}" if self.mi is True else None,
                    h1=f"{h1_:.3f}",
                    tc1=f"{tc1_:.3f}",
                    h2=f"{h2_:.3f}",
                    tc2=f"{tc2_:.3f}",
                )
                

                
                progress_bar.set_postfix(postfix)
            
        
        return None
    
    def _get_time_features(self, data_df: pd.DataFrame)-> pd.DataFrame:
        """This function collapses the spatial dimensions as pivots. This allows
        us to only consider time as the input feature."""
        return data_df.to_dataframe().unstack(level=0).reset_index().drop(columns=['lat', 'lon']).dropna()
    
    def _update_results(self, model, year, tc, h, h_time, mi, mi_time, dims):
        """appends new values to the results dataframe."""
        self.results_df = self.results_df.append({
            'model': model,
            'year': year,
            'tc': tc,
            'h': h,
            'h_time': h_time,
            'mi': mi,
            'mi_time': mi_time,
            'dims': dims,
        }, ignore_index=True
        )
        
        if self.results_path is not None:
            self._save_results()
        return self
    
    def _save_results(self):
        """Saves the dataframe to the assigned results path."""
        self.results_df.to_csv(f"{self.results_path}{self.variable}_{self.save_name}.csv")
        return None

In [28]:
# Initialize experiment
short_decade_exp = ClimateEntropy(
    save_path=f"{results_path}", 
    variable='mslp',
    save_name='era_ncep'
)

# run experiment (shorter decade)
short_decade_exp.run_experiment(era5_data_regrid, ncep_data)

100%|██████████| 41/41 [1:43:03<00:00, 150.83s/it, year=2019, mi=4.766, h1=-1.313, tc1=9.551, h2=-3.049, tc2=13.532] 


In [ ]:
# extract results
results_df = short_decade_exp.results_df

#### ERA5 vs CMIP5

In [ ]:
2006-01-16, 2025-12-16, 1979-01-01, 2019-07-01

In [28]:
# Initialize experiment
short_decade_exp = ClimateEntropy(
    save_path=f"{results_path}", variable='mslp', save_name='era_cmip',
    mi=True
)

# run experiment (shorter decade)
short_decade_exp.run_experiment(
    era5_data_regrid.sel(time=slice('2006-01-16', '2019-07-01')), 
    cmip5_data_regrid.sel(time=slice('2006-01-16', '2019-07-01'))
)

100%|██████████| 14/14 [35:08<00:00, 150.61s/it, year=2019, mi=3.290, h1=-1.509, tc1=9.747, h2=-0.807, tc2=8.142]  


#### NCEP vs CMIP5

In [12]:
# Initialize experiment
short_decade_exp = ClimateEntropy(
    save_path=f"{results_path}", variable='mslp', save_name='ncep_cmip',
    mi=True
)

# run experiment (shorter decade)
short_decade_exp.run_experiment(
    ncep_data.sel(time=slice('2006-01-16', '2019-07-01')), 
    cmip5_data_regrid.sel(time=slice('2006-01-16', '2019-07-01'))
)

100%|██████████| 14/14 [35:04<00:00, 150.30s/it, year=2019, mi=3.552, h1=-1.495, tc1=9.760, h2=-0.807, tc2=8.142]  
